Fitness Fatigue Models Illustrative Code
=========================================

This is a Kaggle Notebook with dependencies on data and scripts hosted on Kaggle.com. To run or edit this notebook, please visit the latest [Kaggle version](https://www.kaggle.com/baogorek/fitness-fatigue-models-illustrative-code).

## Dependencies
 - [example_loads.csv](../input/example-training-loads), a data set of example training loads put together by ... to ...
 - [ffmfunctions.R](../usr/lib/ffmfunctions/ffmfunctions.R), an R script containing functions relevant to the Fitness Fatigue model and variations discussed in our review papers
 

In [ ]:
source("../usr/lib/ffmfunctions/ffmfunctions.R")
example_loads <- read.csv('../input/example-training-loads/example_loads.csv')

set.seed(523445)

## Unfinished Work - To Be Deleted when finished

This section will eventually be deleted. It contains the TODO list of items**
### Notes from 2020-12-1 Meeting

- **DONE** Hard code Kappa to 100, we want max training to be near 100, to reflect the untransformed value being near 100 (which that's not the case now either)
- **DONE** Bring in Ben's training impulses
- **DONE** (see below) Investigate use of Gif for gradient descent model fitting
- **DONE** Renaming
    + xi should be sigma
    + eta_n should be epsilon_n
- Link Notebook to Github repo, so that we have a single source of truth for all code
- Need writeup on the starting values using the grid of time constants
    
  
### Other notes on the Kalman Filter
- Kalman - why is M_0 changing during iterations?

- Kalman Filter - Paul's suggestion: visualize how it adjusts, compared to the standard FFM

- Check to make sure initialize_from_df regression coefficients are in the right order for fitness, fatigue

- What is this printout in the fitting function?
 ```
[1] "filtered$df will have variable 'y_hat'"
6.2s
24
[1] "filtered$X is a matrix with fitness (column 1) and fatigue (column 2)"
7.5s
25
``` 

> ### Notes and Answers
Answer to the gif question [here](https://www.kaggle.com/getting-started/100824)
Use img src tag (take it out of the triple backticks):
```
<img src="https://media.giphy.com/media/10LKovKon8DENq/giphy.gif">
```

Plotting logic out of increase likelihood by gradient

```
  plot(df$y ~ df$t,
	    main = paste("Gradient Descent: Pred vs Observed (blue), R-squared:",
                     round(cor(pred$y_hat, df$y) ^ 2, 3)))
  points(pred$y_hat ~ df$t, col = 'blue')
```

In [ ]:
# The Training Plan -----------------------------------------------------------

# "upper body", "lower body", "synthetic"
training_type <- "upper body" 

if (training_type == "synthetic") {
  w <- rep(c(seq(10, 50), rep(20, 14)), 5)
  w <- c(w, rep(0, 100), w)  #  Adding long rest!
} else if (training_type == "upper body") {
  w <- as.numeric(example_loads$tl_upper_fitness)
} else if (training_type == "lower body") {
  w <- as.numeric(example_loads$tl_lower_fitness)
}

plot(w, main = "Training impulses for this demonstration", xlab = "time")

## The Basic FFM

This is the basic 5-parameter Fitness Fatigue Model.

$$
p_n = p_0 + k_g \sum_{i=1}^{n-1} \omega_i \cdot \exp^{-\frac{(n-i)}{\tau_g}}
          - k_h \sum_{i=1}^{n-1} \omega_i \cdot \exp^{-\frac{(n-i)}{\tau_h}}
$$

In [ ]:
# Basic FFM -------------------------------------------------------------------
ffm_basic <- create_ffm_model(p_0 = 400, k_g = 1, k_h = 3, tau_g = 60,
                              tau_h = 15, sigma = 20)

print(ffm_basic)

In [ ]:
df <- simulate(ffm_basic, w)

# Predictions with true parameters
pred_true_df <- make_predictions(ffm_basic, w)
plot(df$y)
points(pred_true_df$y_hat, col = 'blue')

In [ ]:

# Estimating basic model, first get starting values from data set
ffm_from_data <- initialize_ffm_from_data(df, tau_g_seq = c(10, 50, 90),
                                          tau_h_seq = c(5, 10, 20))
print(ffm_from_data)

# Predictions with true parameters
pred_starting_df <- make_predictions(ffm_from_data, w)

In [ ]:
# One-shot maximum likelihood using L-BFGS-B
ffm_ml <- maximize_likelihood(ffm_from_data, df)
print(ffm_ml)

### Gradient Descent with the 5-parameter FFM
The gradient vector of the sum of squared residuals with respect to the fitness fatigue model is analytically tractible and may be used in a gradient descent algorithm. This method is included for comparison with the L-BFGS-G method used in R's base `optim` function. While it is possible to reduce the error dramatically from an intial condition, scaling is important.

This procedure normalizes the gradient, then applies individual scaling of the parameters before applying the usual $\lambda$ tuning rate parameter. It may be necessary to run the algorithm multiple times with different scalings and values of $\lambda$.

In [ ]:
# Demonstration - to get feel for Gradient Descent

ffm_close <- create_ffm_model(p_0 = 385, k_g = .5, k_h = 2.5, tau_g = 52,
                              tau_h = 12, sigma = 15)


ffm_gd <- increase_likelihood_by_gradient(ffm_close, df, reps = 5, lambda = .001)

ffm_gd <- increase_likelihood_by_gradient(ffm_gd, df, reps = 3000, lambda = .01,
                                          thin = 100, parscale = c(.01, 4, 1, .05, .25))

ffm_gd <- increase_likelihood_by_gradient(ffm_gd, df, reps = 1500, lambda = .0001,
                                          thin = 50, parscale = c(.02, 2, .5, .05, .1))


print(ffm_gd)

FFM with initial values
====

In [ ]:
ffm_add_initial <- create_ffm_model(p_0 = 400, k_g = 1, k_h = 3, tau_g = 60,
                                    tau_h = 15, sigma = 20, q_g = 500, q_h = 250)
print(ffm_add_initial)

In [ ]:
df <- simulate(ffm_add_initial, w)
head(df, 5)  # Look for fitness and fatigue to be around their starting values

In [ ]:
# See the initial values at work
plot(df$fitness, main = "fitness", xlab = "time")
plot(df$fatigue, main = "fatigue", xlab = "time")

In [ ]:
ffm_from_data <- initialize_ffm_from_data(df)

In [ ]:
ffm_ml <- maximize_likelihood(ffm_from_data, df, tune_initial = TRUE)
print(ffm_ml)

In [ ]:
# Predictions with estimated model
pred_df <- make_predictions(ffm_ml, w)
plot(df$y, main = "predictions vs data for initial values model")
points(pred_df$y_hat, col = 'red')

# Estimate VDR Parameters (and initial values)

Fatigue effects can become very large, very fast in the VDR model. Try setting $k_h$ to 3 as in the above examples, and look at the shape of the simulated data.

In [ ]:
ffm_vdr <- create_ffm_model(p_0 = 400, k_g = 1, k_h = 1.5, tau_g = 60,
                            tau_h = 15, sigma = 20,
                            tau_h2 = 3, 
                            q_g = 300, q_h = 250)
print(ffm_vdr)

In [ ]:
df <- simulate(ffm_vdr, w)
head(df, 5)

In [ ]:
# Fatigue can get pretty large with tau_h and tau_h2
plot(df$fitness, main = "Fitness with VDR")
plot(df$fatigue, main = "Fatigue with VDR")

In [ ]:
# Predictions with true parameters
pred_true_df <- make_predictions(ffm_vdr, w)
plot(df$y, main = "VDR simulated and predictions with true values")
points(pred_true_df$y_hat, col = 'blue')

In [ ]:
# Specify tau_h2_seq for data-driven VDR starting values
ffm_from_data <- initialize_ffm_from_data(df,
                                          tau_g_seq = c(10, 50, 80),
                                          tau_h_seq = c(5, 10, 20),
                                          tau_h2_seq = c(1, 2, 5, 10, 15))

In [ ]:
# One-shot maximum likelihood using L-BFGS-B
ffm_ml <- maximize_likelihood(ffm_from_data, df, tune_initial = TRUE, tune_vdr = TRUE)
print(ffm_ml)

In [ ]:
pred_df <- make_predictions(ffm_ml, w)
plot(df$y, main = "predictions in VDR & initial based on ML Fit")
points(pred_df$y_hat, col = 'red')

Hill coefficient estimation
====

In [ ]:
ffm_hill <- create_ffm_model(p_0 = 400, k_g = 1, k_h = 3, tau_g = 60,
                             tau_h = 15, sigma = 20,
                             gamma = 2, delta = 10, kappa = 100)
print(ffm_hill)

In [ ]:
w_hill <- get_hill_transformed_training(ffm_hill, w)
plot(w_hill ~ w, main = "Simulation-specified Hill Transformation")

In [ ]:
df <- simulate(ffm_hill, w)
head(df)

In [ ]:
# Predictions with true model
pred_df <- make_predictions(ffm_hill, w)
plot(df$y, main = "Predictions with True Model")
points(pred_df$y_hat, col = 'red')

In [ ]:
# Specify delta_seq and gamma_seq for data-driven hill starting values
ffm_from_data <- initialize_ffm_from_data(df,
                                          tau_g_seq = c(10, 50, 80),
                                          tau_h_seq = c(5, 10, 20),
                                          delta_seq = c(.3, 1, 1.5, 5, 20),
                                          gamma_seq = c(.3, 1, 2, 5, 20))
ffm_from_data

In [ ]:
# One-shot maximum likelihood using L-BFGS-B
ffm_ml <- maximize_likelihood(ffm_from_data, df, tune_hill = TRUE)
print(ffm_ml)

In [ ]:
# Hill transformation analysis
w_hill_ml <- get_hill_transformed_training(ffm_ml, w)
plot(w_hill ~ w, 
     main = "Hill transformation - True (black) & Est (blue )")
points(w_hill_ml ~ w, col = 'blue')

In [ ]:
# Predictions with estimated model
pred_df <- make_predictions(ffm_ml, w)
plot(df$y, main = "Predictions with Hill-estimated mode")
points(pred_df$y_hat, col = 'red')

"The Works": VDR, Hill and Initial Value estimation
==========

In [ ]:
ffm_the_works <- create_ffm_model(p_0 = 400, k_g = 1, k_h = .9, tau_g = 60,
                                  tau_h = 15, sigma = 20, tau_h2 = 3,
                                  gamma = 2, delta = 10,
                                  q_g = 300, q_h = 250)
print(ffm_the_works)

In [ ]:
w_hill_the_works <- get_hill_transformed_training(ffm_the_works, w)
plot(w_hill_the_works ~ w, main = "Hill transformation")

In [ ]:
df <- simulate(ffm_the_works, w)
head(df)

In [ ]:
plot(df$fitness, main = "fitness", xlab = "time")
plot(df$fatigue, main = "fatigue", xlab = "time")

In [ ]:
pred_df <- make_predictions(ffm_the_works, w)
plot(df$y, main = "Simulated data and true model predictions")
points(pred_df$y_hat, col = 'red')

In [ ]:
ffm_from_data <- initialize_ffm_from_data(df,
                                          tau_g_seq = c(10, 50, 80),
                                          tau_h_seq = c(5, 10, 20),
                                          tau_h2_seq = c(1, 2, 5, 10),
                                          delta_seq = c(.3, 1, 1.5, 5, 20),
                                          gamma_seq = c(.3, 1, 2, 5, 20))

In [ ]:
ffm_ml <- maximize_likelihood(ffm_from_data, df, tune_initial = TRUE,
                              tune_vdr = TRUE, tune_hill = TRUE)
print(ffm_ml)

In [ ]:
# Predictions with estimated model
pred_df <- make_predictions(ffm_ml, w)
plot(df$y, main = "Predictions from ML-estimated 'The Works' model")
points(pred_df$y_hat, col = 'red')

In [ ]:
w_hill_ml <- get_hill_transformed_training(ffm_ml, w)
plot(w_hill_ml ~ w,
     main = "Hill transformation for 'The Works' - True (black) & Est (blue )")